In [ ]:
import numpy as np
import cv2
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from scipy.ndimage.measurements import label
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle

from moviepy.editor import VideoFileClip


%matplotlib inline

### Sliding Window

There are at least two ways to implement a sliding window search. Either use a size-fixed window to silde in different scaled images, or use different sized windows to slide in the same image. I tried both methods. However, only the first method is presented. 

#### Method One (only code)

The following function `slide_window()` takes in an image, start and stop positions in both x and y (imagine a bounding box for the entire search region), window size (x and y dimensions), and overlap fraction (also for both x and y). And it returns a list of bounding boxes for the search windows, which will then be passed to draw draw_boxes() function.

The function `draw_boxes()` takes an image, a list of bounding boxes, and optional color tuple and line thickness as inputs, then draws boxes in that color on the output.

#### Method Two - Part I: Auxiliary Functions

In [ ]:
def convert_color(img, conv='RGB2YCrCb'):
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)

In [ ]:
def get_hog_features(img, 
                     orient, 
                     pix_per_cell, 
                     cell_per_block, 
                     vis=False, 
                     feature_vec=True):
    """
    :param orient: the number of orientations. It is specified as an integer and represents
        the number of orientation bins that the gradient information will be split up into
        in the histogram. Typical values are between 6 and 12 bins.
    :param pixels_per_cell: It is a 2-tuple, which specifies the cell size over which each 
        gradient histogram is computed. 
    :param cell_per_block: It is a 2-tuple, which specifies the local area over which the 
        histogram counts in a given cell will be normalized.
    :return: HOG features and visualization
    """
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, 
                                  orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, 
                                  visualise=vis, 
                                  feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, 
                       orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, 
                       visualise=vis, 
                       feature_vector=feature_vec)
        return features

#### Method Two - Part II: Implementation

Implement a sliding-window technique and use the trained classifier to search for vehicles in images.

In [ ]:
svc = pickle.load(open("saved_svc_YCrCb.p", "rb"))
X_scaler = pickle.load(open("saved_X_scaler_YCrCb.p", "rb"))

In [ ]:
def find_cars(img, 
              scale, 
              svc,
              X_scaler,
              ystart = 400, 
              ystop = 656, 
              orient = 8, 
              pix_per_cell = 8, 
              cell_per_block = 1, 
              spatial_size = (32,32), 
              hist_bins = 32, 
              cspace = "YCrCb", 
              showImage=True):
    """
    This is a function that can extract features using hog sub-sampling
    and make predictions
    """
    # count records the total number of cars detected
    count = 0    
    draw_img = np.copy(img)
    boxes =[]
    
    img_tosearch = img[ystart:ystop,:,:]
    
    #ctrans_tosearch = convert_color(img_tosearch, conv='RGB2YCrCb')
    if cspace != 'RGB':
        if cspace == 'HSV':
            ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2HSV)
        elif cspace == 'LUV':
            ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2LUV)
        elif cspace == 'HLS':
            ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2HLS)
        elif cspace == 'YUV':
            ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2YUV)
        elif cspace == 'YCrCb':
            ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2YCrCb)
    else: ctrans_tosearch = np.copy(img_tosearch)  
                    
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, 
                                     (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]
    
    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell)-1
    nyblocks = (ch1.shape[0] // pix_per_cell)-1 
    nfeat_per_block = orient*cell_per_block**2
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell)
    
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
                               
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].reshape(1, -1)   
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].reshape(1, -1)   
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].reshape(1, -1)   
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            test_features = X_scaler.transform(hog_features).reshape(1, -1)   

            test_prediction = svc.predict(test_features)

            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                c1 =  np.random.randint(0, 255)
                c2 =  np.random.randint(0, 255)
                c3 =  np.random.randint(0, 255)
                count+=1
                if showImage==True:
                    cv2.rectangle(draw_img,
                                  (xbox_left, ytop_draw+ystart),
                                  (xbox_left+win_draw,ytop_draw+win_draw+ystart),
                                  (c1,c2,c3),6)              
                    cv2.putText(draw_img,
                                str( count), 
                                (int(xbox_left), int(ytop_draw+ystart)), 
                                cv2.FONT_HERSHEY_SIMPLEX, 
                                1,(255,255,255), 2,  
                                lineType = cv2.LINE_AA)
                else:
                    if count>0:
                        boxes.append(((xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart)))

    if showImage:
        return draw_img, count
    else:
        return boxes, count

#### Experiment: my method  of choosing scales

**Collect Testing Data**

I will not only use the six images from the folder 'test_images'. The testing images and video chuncks are also collected from 'project_video.mp4'. For example, we choose the frames at time 6.5s, 50s, 29s, 30s, 47s, 29.5s. And we cut and save the video between 20s and 26s. 

Code are as follows

In [ ]:
vidcap = cv2.VideoCapture('project_video.mp4')
vidcap.set(cv2.CAP_PROP_POS_MSEC,6500)      # just cue to 6 sec. position
success,image = vidcap.read()
if success:
    cv2.imwrite("./test_images/test7.jpg", image)     # save frame as JPEG file

vidcap.set(cv2.CAP_PROP_POS_MSEC,50000)      # just cue to 50 sec. position
success,image = vidcap.read()
if success:
    cv2.imwrite("./test_images/test8.jpg", image)     # save frame as JPEG file

#vidcap = cv2.VideoCapture('project_video.mp4')
vidcap.set(cv2.CAP_PROP_POS_MSEC,29000)      # just cue to 29 sec. position
success,image = vidcap.read()
if success:
    cv2.imwrite("./test_images/test9.jpg", image)     # save frame as JPEG file

#vidcap = cv2.VideoCapture('project_video.mp4')
vidcap.set(cv2.CAP_PROP_POS_MSEC,30000)      # just cue to 30 sec. position
success,image = vidcap.read()
if success:
    cv2.imwrite("./test_images/test10.jpg", image)     # save frame as JPEG file

#vidcap = cv2.VideoCapture('project_video.mp4')
vidcap.set(cv2.CAP_PROP_POS_MSEC,47000)      # just cue to 47 sec. position
success,image = vidcap.read()
if success:
    cv2.imwrite("./test_images/test11.jpg", image)     # save frame as JPEG file

#vidcap = cv2.VideoCapture('project_video.mp4')
vidcap.set(cv2.CAP_PROP_POS_MSEC,29500)      # just cue to 29.5 sec. position
success,image = vidcap.read()
if success:
    cv2.imwrite("./test_images/test12.jpg", image)     # save frame as JPEG file
    #cv2.imshow("29sec",image)
    #cv2.waitKey()

In [ ]:
from moviepy.editor import VideoFileClip

extract_video = VideoFileClip("project_video.mp4", audio=False).subclip(20,26)
%time extract_video.write_videofile("test2.mp4")

In [ ]:
from moviepy.editor import VideoFileClip

extract_video = VideoFileClip("project_video.mp4", audio=False).subclip(26,31)
%time extract_video.write_videofile("test2.mp4")

**Test with all possible scales**

In [ ]:
#Test the window classifier in one image
img = mpimg.imread('test_images/test1.jpg')

scales = np.linspace(1.1, 3.4, num=24)
plt.figure(figsize=(12,48))
row = len(scales) // 2 + 1
for i in range(row-1):
    plt.subplot(row, 2, 2*i + 1)
    scale = scales[2*i]
    out_img, count = find_cars(img, 
                               scale, 
                               svc, 
                               X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))
    plt.subplot(row, 2, 2*i + 2)
    scale = scales[2*i + 1]
    out_img, count = find_cars(img, 
                               scale, 
                               svc, 
                               X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))

plt.savefig("./output_images/scales_test1.jpg")

In [ ]:
img = mpimg.imread('test_images/test3.jpg')

scales = np.linspace(1.1, 3.4, num=24)
plt.figure(figsize=(12,48))
row = len(scales) // 2 + 1
for i in range(row-1):
    plt.subplot(row, 2, 2*i + 1)
    scale = scales[2*i]
    out_img, count = find_cars(img, 
                           scale, 
                           svc, 
                           X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))
    plt.subplot(row, 2, 2*i + 2)
    scale = scales[2*i + 1]
    out_img, count = find_cars(img, 
                           scale, 
                           svc, 
                           X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))
    
plt.savefig("./output_images/scales_test3.jpg")

In [ ]:
img = mpimg.imread('test_images/test4.jpg')

scales = np.linspace(1.1, 3.4, num=24)
plt.figure(figsize=(12,48))
row = len(scales) // 2 + 1
for i in range(row-1):
    plt.subplot(row, 2, 2*i + 1)
    scale = scales[2*i]
    out_img, count = find_cars(img, 
                           scale, 
                           svc, 
                           X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))
    plt.subplot(row, 2, 2*i + 2)
    scale = scales[2*i + 1]
    out_img, count = find_cars(img, 
                           scale, 
                           svc, 
                           X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))

plt.savefig("./output_images/scales_test4.jpg")

In [ ]:
img = mpimg.imread('test_images/test6.jpg')

scales = np.linspace(1.1, 3.4, num=24)
plt.figure(figsize=(12,48))
row = len(scales) // 2 + 1
for i in range(row-1):
    plt.subplot(row, 2, 2*i + 1)
    scale = scales[2*i]
    out_img, count = find_cars(img, 
                           scale, 
                           svc, 
                           X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))
    plt.subplot(row, 2, 2*i + 2)
    scale = scales[2*i + 1]
    out_img, count = find_cars(img, 
                           scale, 
                           svc, 
                           X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))

plt.savefig("./output_images/scales_test6.jpg")

In [ ]:
img = mpimg.imread('test_images/test7.jpg')

scales = np.linspace(1.1, 3.4, num=24)
plt.figure(figsize=(12,48))
row = len(scales) // 2 + 1
for i in range(row-1):
    plt.subplot(row, 2, 2*i + 1)
    scale = scales[2*i]
    out_img, count = find_cars(img, 
                           scale, 
                           svc, 
                           X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))
    plt.subplot(row, 2, 2*i + 2)
    scale = scales[2*i + 1]
    out_img, count = find_cars(img, 
                           scale, 
                           svc, 
                           X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))

plt.savefig("./output_images/scales_test7.jpg")

In [ ]:
img = mpimg.imread('test_images/test8.jpg')

scales = np.linspace(1.1, 3.4, num=24)
plt.figure(figsize=(12,48))
row = len(scales) // 2 + 1
for i in range(row-1):
    plt.subplot(row, 2, 2*i + 1)
    scale = scales[2*i]
    out_img, count = find_cars(img, 
                           scale, 
                           svc, 
                           X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))
    plt.subplot(row, 2, 2*i + 2)
    scale = scales[2*i + 1]
    out_img, count = find_cars(img, 
                           scale, 
                           svc, 
                           X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))

plt.savefig("./output_images/scales_test8.jpg")

In [ ]:
img = mpimg.imread('test_images/test9.jpg')

scales = np.linspace(1.1, 3.4, num=24)
plt.figure(figsize=(12,48))
row = len(scales) // 2 + 1
for i in range(row-1):
    plt.subplot(row, 2, 2*i + 1)
    scale = scales[2*i]
    out_img, count = find_cars(img, 
                           scale, 
                           svc, 
                           X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))
    plt.subplot(row, 2, 2*i + 2)
    scale = scales[2*i + 1]
    out_img, count = find_cars(img, 
                           scale, 
                           svc, 
                           X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))

plt.savefig("./output_images/scales_test9.jpg")

In [ ]:
img = mpimg.imread('test_images/test10.jpg')

scales = np.linspace(1.1, 3.4, num=24)
plt.figure(figsize=(12,48))
row = len(scales) // 2 + 1
for i in range(row-1):
    plt.subplot(row, 2, 2*i + 1)
    scale = scales[2*i]
    out_img, count = find_cars(img, 
                           scale, 
                           svc, 
                           X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))
    plt.subplot(row, 2, 2*i + 2)
    scale = scales[2*i + 1]
    out_img, count = find_cars(img, 
                           scale, 
                           svc, 
                           X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))

plt.savefig("./output_images/scales_test10.jpg")

In [ ]:
img = mpimg.imread('test_images/test11.jpg')

scales = np.linspace(1.1, 3.4, num=24)
plt.figure(figsize=(12,64))
row = len(scales) // 2 + 1
for i in range(row-1):
    plt.subplot(row, 2, 2*i + 1)
    scale = scales[2*i]
    out_img, count = find_cars(img, 
                           scale, 
                           svc, 
                           X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))
    plt.subplot(row, 2, 2*i + 2)
    scale = scales[2*i + 1]
    out_img, count = find_cars(img, 
                           scale, 
                           svc, 
                           X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))

plt.savefig("./output_images/scales_test11.jpg")

In [ ]:
img = mpimg.imread('test_images/test12.jpg')

scales = np.linspace(1.1, 3.4, num=24)
plt.figure(figsize=(12,64))
row = len(scales) // 2 + 1
for i in range(row-1):
    plt.subplot(row, 2, 2*i + 1)
    scale = scales[2*i]
    out_img, count = find_cars(img, 
                           scale, 
                           svc, 
                           X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))
    plt.subplot(row, 2, 2*i + 2)
    scale = scales[2*i + 1]
    out_img, count = find_cars(img, 
                           scale, 
                           svc, 
                           X_scaler)
    plt.imshow(out_img)
    plt.title("scale = {}, count = {}".format(scale, count))
    plt.savefig("./output_images/choose_scales2.jpg")

plt.savefig("./output_images/scales_test12.jpg")

#### Experiment: Conclusion

With above experiments results, one can obtain a table, with columns recording different scales and row corresponding to different testing images. It is not too hard to pick out some 'reasonable' scales for our purpose. Here, I set the scales that will used later as [1.1, 1.4, 1.8, 2.3, 2.6, 2.9].

In [ ]:
def find_cars_multiple(img, 
                       svc, 
                       X_scaler,
                       ystart = 400, 
                       ystop = 656, 
                       orient = 8, 
                       pix_per_cell = 8, 
                       cell_per_block = 1, 
                       spatial_size = (32,32), 
                       hist_bins = 32, 
                       cspace = "YCrCb"):
    scales = [1.1, 1.4, 1.8, 2.3, 2.6, 2.9]
    c=0
    bbox = []           
    for scale in scales:
        c += 1
        #The first four of scales is valid for the upper half of image
        if c < 5:
            ystartaux = ystart
            ystopaux = int(ystart + (ystop-ystart)*3/4)
        else:
            ystartaux = int(ystart + (ystop-ystart)/4)
            ystopaux = ystop

        box, count = find_cars(img = img, 
                               scale = scale,
                               svc = svc,
                               X_scaler = X_scaler,
                               ystart = ystartaux, 
                               ystop = ystopaux, 
                               orient = orient, 
                               pix_per_cell = pix_per_cell, 
                               cell_per_block = cell_per_block, 
                               spatial_size = spatial_size, 
                               hist_bins = hist_bins, 
                               cspace = cspace, 
                               showImage = False)
        if count>0:
            for b in box:
                bbox.append(b)
    return bbox

#### Heat Map

In [ ]:
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
    # Return updated heatmap
    return heatmap # Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(labels, heatmap,img=None):
    # Iterate through all detected cars
    b_heat=[] #((x1,y1),(x2,y2))

    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        highv = np.sum(heatmap[np.min(nonzeroy):np.max(nonzeroy), np.min(nonzerox):np.max(nonzerox)])
        # Draw the box on the image
        if not img is None:
            cv2.rectangle(img, bbox[0], bbox[1], (0,0,200), 3)
        
        b_heat.append(((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy))))

    # Return the image
    if not img is None:
        return img,b_heat
    else:
        return b_heat

def heatmap(image, box_list, threshold=1, showImg = True):
    
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    # Add heat to each box in box list
    heat = add_heat(heat,box_list)

    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,threshold)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    
    if showImg ==  True:
        draw_img,lbl_heat = draw_labeled_bboxes(labels, heatmap, np.copy(image))
        return draw_img, heatmap, lbl_heat
    else:
        lbl_heat = draw_labeled_bboxes(labels,heatmap)
        return heatmap, lbl_heat

#### Examples

In [ ]:
image = mpimg.imread('./test_images/test1.jpg')

box_list = find_cars_multiple(img = image, 
                           svc = svc,
                           X_scaler = X_scaler,
                           ystart = 400, 
                           ystop = 656, 
                           orient = 8, 
                           pix_per_cell = 8, 
                           cell_per_block = 1, 
                           spatial_size = (32,32), 
                           hist_bins = 32, 
                           cspace = "YCrCb") 

thres = 1
draw_img, heat_map, lbl_heat = heatmap(image, box_list, thres, True)

fig = plt.figure(figsize=(12,8))
plt.subplot(121)
plt.imshow(draw_img)
plt.title('Car Positions')
plt.subplot(122)
plt.imshow(heat_map, cmap='hot')
plt.title('Heat Map')

plt.savefig("./output_images/heatmap1.jpg")

In [ ]:
image = mpimg.imread('./test_images/test12.jpg')

box_list = find_cars_multiple(img = image, 
                           svc = svc,
                           X_scaler = X_scaler,
                           ystart = 400, 
                           ystop = 656, 
                           orient = 8, 
                           pix_per_cell = 8, 
                           cell_per_block = 1, 
                           spatial_size = (32,32), 
                           hist_bins = 32, 
                           cspace = "YCrCb") 

thres = 1
draw_img, heat_map, lbl_heat = heatmap(image, box_list, thres, True)

fig = plt.figure(figsize=(12,8))
plt.subplot(121)
plt.imshow(draw_img)
plt.title('Car Positions')
plt.subplot(122)
plt.imshow(heat_map, cmap='hot')
plt.title('Heat Map')

plt.savefig("./output_images/heatmap12.jpg")

In [ ]:
image = mpimg.imread('./test_images/test5.jpg')

box_list = find_cars_multiple(img = image, 
                           svc = svc,
                           X_scaler = X_scaler,
                           ystart = 400, 
                           ystop = 656, 
                           orient = 8, 
                           pix_per_cell = 8, 
                           cell_per_block = 1, 
                           spatial_size = (32,32), 
                           hist_bins = 32, 
                           cspace = "YCrCb") 

thres = 1
draw_img, heat_map, lbl_heat = heatmap(image, box_list, thres, True)

fig = plt.figure(figsize=(12,8))
plt.subplot(121)
plt.imshow(draw_img)
plt.title('Car Positions')
plt.subplot(122)
plt.imshow(heat_map, cmap='hot')
plt.title('Heat Map')
plt.savefig("./output_images/heatmap5.jpg")

### The Pipeline

In [ ]:
svc = pickle.load(open("saved_svc_YCrCb.p", "rb")) 
X_scaler = pickle.load(open("saved_X_scaler_YCrCb.p", "rb")) 
ystart = 400
ystop = 656
orient = 8 
pix_per_cell = 8 
cell_per_block = 1 
spatial_size = (32, 32) 
hist_bins = 32 
cspace = 'YCrCb'


nframes = 10
smooth_thres = 2
smooth_average= 6
bbox_frames =[]
#Initialization of list
for i in range(nframes):
    bbox_frames.append(0)
counter = 0

def process_image(img):
    
    global counter
    global bbox_frames
    
    counter += 1
    countFrame = counter % nframes
    
    #Find rectangles for one image
    bbox1 = find_cars_multiple(img = img, 
                               svc = svc,
                               X_scaler = X_scaler,
                               ystart = ystart,  
                               ystop = ystop, 
                               orient = orient, 
                               pix_per_cell = pix_per_cell, 
                               cell_per_block = cell_per_block, 
                               spatial_size = spatial_size, 
                               hist_bins = hist_bins, 
                               cspace = cspace)

    # Find heatmap single image
    thres = smooth_thres
    heat, bboxHeat = heatmap(img, bbox1, thres, False)
    #dimg, heat, bboxHeat = heatmap(img, bbox1, thres, True)


    # Store the rectangles of the frame
    bbox_frames[countFrame] = bboxHeat
    
    # Sum rectangles of the nframes
    bbox2 = []
    for box in bbox_frames:
        if box != 0:
            for b in box:
                bbox2.append(b)
                
    # Find heatmap of average
    thres = smooth_average
    dimg, heat, bboxHeat2 = heatmap(img, bbox2, thres, True)    

    return dimg

In [ ]:
image = mpimg.imread('test_images/test6.jpg')

plt.figure(figsize=(25,10))
nframes = 1
smooth_thres = 1
smooth_average = 0
bbox_frames=[]
#Inicialization of list
for i in range(nframes):
    bbox_frames.append(0)
counter = 0

img2 = process_image(image)
plt.imshow(img2)
plt.savefig("./output_images/pipeline1.jpg")

In [ ]:
# Import everything needed to edit/save/watch video clips
#from IPython.display import HTML
#import moviepy as mve

In [ ]:
counter = 0
nframes = 26
smooth_thres = 1
smooth_average = 5

bbox_frames=[]
#Inicialization of list
for i in range(nframes):
    bbox_frames.append(0)
counter = 0

output = 'output_test2.mp4'
clip1 = VideoFileClip("test2.mp4", audio=False)

out_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time out_clip.write_videofile(output, audio=False)

In [ ]:
from IPython.display import HTML
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))

In [ ]:
#Create video file pipeline
counter = 0
nframes = 26
smooth_thres = 1
smooth_average = 5

bbox_frames=[]
#Inicialization of list
for i in range(nframes):
    bbox_frames.append(0)
counter = 0

output = 'output_project_video.mp4'
clip1 = VideoFileClip("project_video.mp4", audio=False)

out_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time out_clip.write_videofile(output, audio=False)

In [ ]:
from IPython.display import HTML
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))